# Amazon Bedrock Fundamental

> 이 노트북은, SageMaker Notebook <i><b>conda_python3</b></i> 커널에서 테스트 되었습니다.

## 0. 환경 설정

In [ ]:
# 필요한 라이브러리를 설치합니다.
# !pip install -r ./requirements.txt
%pip install boto3
%pip install botocore

## 1. boto3를 활용한 Bedrock Claude 3 호출 예시 (not LangChain)

#### 1-1. Bedrock 에서 사용 가능한 리스트 확인

In [ ]:
import json
import boto3
from pprint import pprint

# Bedrock Client 생성
bedrock = boto3.client(service_name='bedrock')

# 사용가능한 모델 확인하기
model_list = bedrock.list_foundation_models()
ondemand_models = []
for model_info in model_list["modelSummaries"]:
    if model_info['inferenceTypesSupported'] == ['ON_DEMAND']:
        ondemand_models.append((model_info["modelName"], model_info["modelId"]))

pprint(ondemand_models)

#### 1-2. Claude 3 모델 호출

In [ ]:
# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [ ]:
# 사용할 모델 아이디 작성
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

# Request body 작성
query = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
request_data = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature" : 0,
    "top_k": 250,
    "top_p": 1.0,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
            ],
        }
    ],
}
request_body = json.dumps(request_data)

In [ ]:
# 모델 호출
response = bedrock_runtime.invoke_model(
    body=request_body,
    modelId=modelId,
    contentType="application/json",
    accept="application/json"
)

response_body = json.loads(response['body'].read())
answer = response_body['content'][0]['text']
print(answer)

#### 1-3. Claude 3 모델 스트리밍으로 호출

In [ ]:
# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [ ]:
# 사용할 모델 아이디 작성
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

# Request body 작성
query = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
request_data = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature" : 0,
    "top_k": 250,
    "top_p": 1.0,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
            ],
        }
    ],
}
request_body = json.dumps(request_data)

In [ ]:
# Streaming 으로 모델 호출
response = bedrock_runtime.invoke_model_with_response_stream(
    body=request_body,
    modelId=modelId,
    contentType="application/json",
    accept="application/json"
)

stream = response.get('body')
answer = ""
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            if "delta" in chunk_obj:                    
                delta = chunk_obj['delta']
                if "text" in delta:
                    text=delta['text'] 
                    print(text, end="")
                    answer+=str(text) 

print(answer)

---

## 2. LangChain을 활용한 Bedrock 사용 예시


In [ ]:
# 필요한 라이브러리 설치
%pip install langchain==0.2.1
%pip install langchain_aws==0.1.6
%pip install langchain_community==0.2.1

#### 2-1. LangChain의 ChatBedrock을 활용한 간단한 코드 예시

In [ ]:
import langchain
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage
# langchain.debug=True

# ChatBedrock llm 생성
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 512,
        "temperature": 0,
        "top_p": 0.5,
        "top_k": 0,
    }
)

# 입력: Option 1
messages = [
    HumanMessage(
        content="안녕? 만나서 반가워"
    )
]

# 입력: Option 2
# messages = [
#     ("human", "안녕? 만나서 반가워"),
# ]

# 입력: Option 3
# message = "안녕? 만나서 반가워"

# 일반 방식으로 모델 호출
response = llm.invoke(messages)
answer = response.content
print(f"응답:")
print(answer)
print()


# 스트리밍 방식으로 모델 호출
print("스트리밍 응답:")
for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)
print()

langchain.debug=False

#### 2-2. LangChain의 BedrockChat을 활용한 멀티 턴(대화 히스토리 유지) 구현한 코드 예시

In [ ]:
from abc import ABC
from langchain_aws import ChatBedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks.base import BaseCallbackHandler

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_kwargs = {
    "max_tokens": 512,
    "temperature": 0,
    "top_p": 0.5,
    "top_k": 0
}


llm = ChatBedrock(
        region_name="us-west-2",
        model_id=model_id,
        model_kwargs=model_kwargs
    )

memroy_window = 10
conversation_chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(
        k=memroy_window,
        ai_prefix='AI',
        human_prefix='Human'
    ),
    verbose=True
)

# 일반 방식으로 모델 호출
while True:
    input_text = input("Human: ")
    answer = conversation_chain.predict(input=input_text)
    print("AI:", answer)


In [ ]:
# 스트리밍 방식으로 모델 호출
class StreamingHandler(BaseCallbackHandler, ABC):
    """
    Callback handler to stream the generated text to Streamlit.
    """

    def __init__(self) -> None:
        self.text = ""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        """
        Append the new token to the text and update the Streamlit container.
        """
        self.text += token
        print(token, end ="")

llm = ChatBedrock(
        region_name="us-west-2",
        model_id=model_id,
        model_kwargs=model_kwargs,
        streaming=True,
        callbacks=[StreamingHandler()]
    )

memroy_window = 10
conversation_chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(
        k=memroy_window,
        ai_prefix='AI',
        human_prefix='Human'
    ),
    verbose=True
)

while True:
    input_text = input("Human: ")
    answer = conversation_chain.predict(input=input_text)
    print("AI:", answer)